## Set up


In [3]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [4]:
import nest_asyncio

nest_asyncio.apply()

## Load data


In [5]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["dataset.csv"]).load_data()

## Define LLM and Embedding model

In [6]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [19]:
from llama_index.core import Settings # Settings: Là một đối tượng chứa các thiết lập chung mà framework LlamaIndex sử dụng để làm việc với các mô-đun khác
from llama_index.llms.openai import OpenAI # OpenAI: Đây là lớp cung cấp khả năng kết nối và gọi các mô hình ngôn ngữ (LLMs) từ OpenAI, ví dụ như gpt-3.5-turbo hoặc gpt-4
from llama_index.embeddings.openai import OpenAIEmbedding #OpenAIEmbedding: Đây là lớp để sử dụng mô hình nhúng từ OpenAI, giúp chuyển đổi văn bản thành dạng vector số (embeddings) để phục vụ tìm kiếm hoặc các tác vụ khác.

Settings.llm = OpenAI(model="gpt-3.5-turbo") #Settings.llm: Gắn một mô hình xử lý ngôn ngữ (language model) vào framework, ở đây là gpt-3.5-turbo từ OpenAI.
#LlamaIndex sẽ sử dụng mô hình ngôn ngữ này cho mọi tác vụ yêu cầu tạo văn bản hoặc tương tác với AI, như sinh câu trả lời hoặc xử lý câu hỏi


Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002") # Thiết lập mô hình nhúng để chuyển văn bản thành vector số, Ở đây sử dụng mô hình text-embedding-ada-002 từ OpenAI.
# Tìm kiếm thông tin dựa trên ngữ nghĩa.
# So khớp hoặc phân nhóm các văn bản dựa trên độ tương đồng vector.

## Define Summary Index and Vector Index over the Same Data


In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
#SummaryIndex sử dụng mô hình ngôn ngữ để đọc qua nội dung của các nodes, tạo ra các bản tóm tắt. Khi có một truy vấn, nó sẽ dựa vào thông tin tóm tắt để trả lời câu hỏi hoặc cung cấp nội dung cô đọng
vector_index = VectorStoreIndex(nodes)
# Sử dụng một mô hình nhúng để chuyển các nodes thành vector số. Khi nhận được một truy vấn, nó cũng được chuyển thành vector số. Sau đó, quá trình tìm kiếm sẽ dựa trên độ tương đồng giữa các vector.


## Define Query Engines and Set Metadata

In [9]:
# khởi tạo các Query Engines (công cụ truy vấn) từ hai đối tượng summary_index và vector_index
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
) #được dùng để xây dựng công cụ truy vấn cho mục đích tóm tắt.
vector_query_engine = vector_index.as_query_engine()
# công cụ truy vấn liên quan đến xử lý hoặc tìm kiếm dữ liệu theo không gian vector

In [13]:
from llama_index.core.tools import QueryEngineTool

#định nghĩa hai công cụ truy vấn (QueryEngineTool) dựa trên 
# các query engine đã được khởi tạo trước đó (summary_query_engine và vector_query_engine) và thêm mô tả cụ thể để giải thích mục đích sử dụng của từng công cụ

#  Tạo công cụ tóm tắt:  công cụ truy vấn được thiết kế để trả lời các câu hỏi yêu cầu tóm tắt
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to Presight"
    ),
)

# Tạo công cụ tìm kiếm ngữ cảnh: công cụ truy vấn chuyên dụng cho các câu hỏi yêu cầu tìm kiếm thông tin chi tiết từ tài liệu MetaGPT
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the Presight web."
    ),
)

## Define Router Query Engine

In [14]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

#  tạo công cụ truy vấn có thể định tuyến các truy vấn đến các "Query Engine Tool" phù hợp
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(), #Một lớp để chọn công cụ truy vấn (Query Engine Tool) dựa trên trí tuệ nhân tạo ngôn ngữ (LLM).
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True#Khi được bật, chế độ này cung cấp thông tin chi tiết trong quá trình định tuyến và thực thi truy vấn, chẳng hạn như thông báo về việc công cụ nào được chọn.
)

In [15]:
response = query_engine.query("summary presight")
print(str(response))

Selecting query engine 0: The choice mentions summarization questions related to Presight, which aligns with the task of summarizing Presight..
Presight is an AI-powered data platform that is committed to protecting the privacy of its customers and website visitors. The platform collects various types of information such as email addresses, names, phone numbers, addresses, cookies, and usage data for the purpose of providing and improving its services. Users have the right to access, correct, and amend their personal information through the application. Presight uses automated edit checks to ensure data accuracy and integrity. The collected data is used for maintaining the service, notifying users of changes, providing customer support, analyzing for improvements, monitoring usage, and addressing technical issues. Personal information is not retained or used for developing generalized AI models. Data security measures include encryption, regular security audits, and employee training. 

## Tool calling


### Define the Auto-Retrieval Tool

In [16]:
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core.vector_stores import MetadataFilters
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=1,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query,
    description="Useful for retrieving specific context from the Presight web.."
)

In [17]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to Presight"
    ),
)

In [24]:
llm = OpenAI(model="gpt-3.5-turbo")

response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is PRIVACY POLICY?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Privacy policy"}
=== Function Output ===
Presight is dedicated to safeguarding the privacy of its customers and website visitors by explaining how information is collected, used, and disclosed. Personal data collected includes email addresses, names, phone numbers, addresses, cookies, and usage data. Users have the right to access and edit their personal information. Presight uses collected data for various purposes such as maintaining services, notifying users of changes, providing customer support, and improving services. The platform does not retain or use Google User Data for AI/ML model development and ensures data security through encryption and regular security audits. Data retention is based on active account status, and users are responsible for providing accurate information. The privacy policy is periodically updated, and users can control cookie usage through browser settings. Third-party websites

In [26]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
   "What is a summary of the paper?",
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "paper"}
=== Function Output ===
The entity is committed to using personal information only for the purposes identified in its privacy policy.
